In [4]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


optimal method using pulp library

In [2]:


import os
import pulp

def read_gap_instances(file_path):
    with open(file_path, "r") as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]
    
    index = 0
    num_instances = int(lines[index])
    index += 1

    def read_matrix(rows, cols):
        nonlocal index
        matrix = []
        while len(matrix) < rows:
            row_values = []
            while len(row_values) < cols:
                row_values.extend(map(int, lines[index].split()))
                index += 1
            matrix.append(row_values[:cols])
        return matrix

    instances = []
    for _ in range(num_instances):
        num_servers, num_users = map(int, lines[index].split())
        index += 1

        utility = read_matrix(num_servers, num_users)
        vm_allocation = read_matrix(num_servers, num_users)
        server_capacity = list(map(int, lines[index].split()))
        index += 1

        instances.append((utility, vm_allocation, server_capacity))
    
    return instances

def solve_gap_optimally(utility, vm_allocation, server_capacity, instance_id, file_name):
    num_servers = len(utility)
    num_users = len(utility[0])

    problem = pulp.LpProblem(f"GAP_Instance_{instance_id}_from_{file_name}", pulp.LpMaximize)

    # Binary Decision Variables
    x = [[pulp.LpVariable(f"x_{i}_{j}_{instance_id}", cat='Binary') for j in range(num_users)] for i in range(num_servers)]

    # Objective Function
    problem += pulp.lpSum(utility[i][j] * x[i][j] for i in range(num_servers) for j in range(num_users))

    # Constraints
    for j in range(num_users):
        problem += pulp.lpSum(x[i][j] for i in range(num_servers)) == 1

    for i in range(num_servers):
        problem += pulp.lpSum(vm_allocation[i][j] * x[i][j] for j in range(num_users)) <= server_capacity[i]

    problem.solve()

    # Prepare output
    assignments = [-1] * num_users
    for i in range(num_servers):
        for j in range(num_users):
            if pulp.value(x[i][j]) == 1:
                assignments[j] = i

    total_utility = pulp.value(problem.objective)

    # Output
    print(f"\nProcessing Instance {instance_id} from {file_name}")
    print(f"Task Assignments: {assignments}")
    print(f"Total Optimal Utility: {total_utility}")

if __name__ == "__main__":
    folder_path = r"C:\Users\MD KAIF\Desktop\lab_assignments\ec_lab\gap_dataset_files"
    file_names = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    file_names.sort(key=lambda f: int(f.split('gap')[1].split('.txt')[0]))  # Ensure proper order: gap1, gap2...

    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        instances = read_gap_instances(file_path)

        for instance_id, (utility, vm_allocation, server_capacity) in enumerate(instances, start=1):
            solve_gap_optimally(utility, vm_allocation, server_capacity, instance_id, file_name)



Processing Instance 1 from gap1.txt
Task Assignments: [1, 1, 3, 2, 0, 4, 0, 1, 0, 3, 3, 3, 0, 4, 2]
Total Optimal Utility: 336.0

Processing Instance 2 from gap1.txt
Task Assignments: [3, 0, 0, 2, 2, 0, 2, 1, 3, 1, 2, 1, 3, 4, 4]
Total Optimal Utility: 327.0

Processing Instance 3 from gap1.txt
Task Assignments: [1, 4, 4, 3, 2, 3, 4, 0, 1, 1, 2, 0, 0, 3, 0]
Total Optimal Utility: 339.0

Processing Instance 4 from gap1.txt
Task Assignments: [2, 4, 1, 4, 0, 0, 4, 1, 0, 3, 2, 2, 3, 4, 1]
Total Optimal Utility: 341.0

Processing Instance 5 from gap1.txt
Task Assignments: [1, 3, 0, 3, 4, 1, 4, 3, 2, 2, 0, 1, 0, 4, 4]
Total Optimal Utility: 326.0

Processing Instance 1 from gap2.txt
Task Assignments: [2, 4, 0, 3, 2, 0, 1, 3, 0, 1, 4, 4, 2, 4, 3, 1, 2, 3, 1, 0]
Total Optimal Utility: 434.0

Processing Instance 2 from gap2.txt
Task Assignments: [4, 2, 4, 3, 0, 2, 0, 3, 2, 1, 1, 1, 1, 4, 4, 3, 3, 3, 2, 0]
Total Optimal Utility: 436.0

Processing Instance 3 from gap2.txt
Task Assignments: [3, 4